See API documentation for full set of search parameters:

https://api-staging.terramatch.org/research-service/documentation/api#/default/ 

In [1]:
import requests
import yaml
import json
import pandas as pd

In [2]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [3]:
# API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

In [9]:
# Define search parameters
# import project ids using csv
params = {'projectId[]': '389aad5b-6577-4cea-bf9f-446dcfd94966',
    'polygonStatus[]':'approved',
          'includeTestProjects': 'false',
          'page[size]': '100'
          }

In [5]:
def pull_tm_api_data(url, headers, params):
    results = []
    last_record = ''
    while True:
    # send request
        response = requests.get(url, headers=headers,params=params)
        # check status code
        if response.status_code != 200:
            raise ValueError('Request failed with status code ' + str(response.status_code))
        # parse response data
        for idx in range(0, response.json()['meta']['page']['total']):
            data = response.json()['data'][idx]['attributes']
            data['poly_id'] =response.json()['data'][idx]['meta']['page']['cursor']
        # add results to list
            results.append(data)
            if idx == ((response.json()['meta']['page']['total']) - 1):
                new_last_record = response.json()['data'][idx]['meta']['page']['cursor'] # will there be a scenario where this local variable isnt instantiated?
        # check if there are more pages
        if (last_record != new_last_record):
            last_record = new_last_record
            params['page[after]'] =last_record
        else:
            break
    return results

In [6]:
project_results = pull_tm_api_data(staging_url, headers, params)

In [ ]:
project_df = pd.DataFrame(project_results)
project_df

## Decision Tree Edits

In [7]:
import requests
import yaml
import json
import pandas as pd
from tqdm import tqdm

In [4]:
# Set up token access
auth_path = '../secrets.yaml'

with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

# API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # use for testing queries

In [5]:
def pull_tm_api_data(url, headers, project_ids):
    '''
    edits to the above function include:
        iterating through list of project ids within func so output is a df with 
        multiple projects
        add project id as a column to support maxar metadata request
        update to last record variable
        added tqdm progress bar 
        
    '''
    results = []
    with tqdm(total=len(project_ids), desc="Processing Projects", unit="project") as progress_bar:
        for project_id in project_ids:
            # Set parameters with the current project ID
            params = {
                'projectId[]': project_id,
                'polygonStatus[]': 'approved',
                'includeTestProjects': 'false',
                'page[size]': '100'
            }

            last_record = ''
            new_last_record = None  # Ensure it's defined before use

            while True:
                # Send request
                response = requests.get(url, headers=headers, params=params)

                # Check status code
                if response.status_code != 200:
                    raise ValueError(f'Request failed for project {project_id} with status code {response.status_code}')
                
                response_json = response.json()
                total_records = response_json['meta']['page']['total']

                # Parse response data
                if total_records == 0:
                    break  # Exit if no data is available

                for idx in range(total_records):
                    data = response_json['data'][idx]['attributes']
                    data['poly_id'] = response_json['data'][idx]['meta']['page']['cursor']
                    data['project_id'] = project_id 
                    results.append(data)

                    # Assign the last cursor only if there are records
                    if idx == (total_records - 1):
                        new_last_record = response_json['data'][idx]['meta']['page']['cursor']

                # Check if there are more pages
                if new_last_record and last_record != new_last_record:
                    last_record = new_last_record
                    params['page[after]'] = last_record
                else:
                    break  # Exit pagination if no new cursor is found

            progress_bar.update(1) 
    return results


In [25]:
# make the request for x projects
full = pd.read_csv('../projects_all_approved_202501091214.csv')
full = full[(full.framework_key == 'terrafund-landscapes') | (full.framework_key == 'terrafund')]
full.framework_key.value_counts()

terrafund               108
terrafund-landscapes     99
Name: framework_key, dtype: int64

In [21]:
full.head()

,project_id,project_name,organization_name,organisation_id,country,framework_key,description,status
0,c462918b-47f7-4ed5-99e0-7fec6e342036,"""Nakuru Eco-Reforestation Project""",NaN,1382,KE,terrafund-landscapes,MAIN ACTIVITIES\nJAN - FEB 2024- TREE NURSERY...,approved
1,caae56f9-0bb6-45a2-9d77-ff088b085917,0726 project,NaN,6283,BR,terrafund,org 0726,approved
2,c004619e-c1aa-4f7f-b56b-c8f9b4385d4e,1,NaN,1582,AL,terrafund,1,approved
4,6083e1cf-a636-4c64-9253-ac86cd08f5d7,3SC Production 2.3,NaN,3279,AF,terrafund,3SC Production 2.3,approved
6,617601e0-9839-49fd-b48e-6c07404e7140,Afram Headwaters Restoration Initiative (AHRI),NaN,1358,GH,terrafund-landscapes,1.\tSite Reconnaissance – which has already be...,approved


In [26]:
ids = list(set(full.project_id))
# ids = ids[:50]
# ids

In [27]:
project_results = pull_tm_api_data(staging_url, headers, ids)

Processing Projects: 100%|████████████████████████████████████████████████████████████████████| 207/207 [07:32<00:00,  2.19s/project]


,name,status,siteId,geometry,plantStart,plantEnd,practice,targetSys,distr,numTrees,calcArea,indicators,establishmentTreeSpecies,reportingPeriods,poly_id,project_id
0,R12W 11,approved,60aa51dc-65f1-4b1d-be97-a7e962201819,"{'type': 'Polygon', 'coordinates': [[[32.64296...",2023-02-22,2023-03-31,tree-planting,agroforest,full,0.0,0.116644,[],[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",f8575cf2-9b8e-4e93-90de-667b15f8f0c0,6b2dc32e-4414-48e0-9472-ebb49a766c28
1,R29W11,approved,60aa51dc-65f1-4b1d-be97-a7e962201819,"{'type': 'Polygon', 'coordinates': [[[32.64465...",2024-01-28,2024-09-28,tree-planting,agroforest,full,300.0,1.694570,[],[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",02e33916-6380-4a18-81e7-97d29db84cb5,6b2dc32e-4414-48e0-9472-ebb49a766c28
2,R39W5,approved,990421d0-2287-4a8c-824a-cbf732199c33,"{'type': 'Polygon', 'coordinates': [[[32.83510...",2024-04-01,2024-07-31,tree-planting,agroforest,full,NaN,0.046533,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2023-01-31T00:00:00.000Z', 'submit...",05733383-5ddb-4e2f-8ba9-e5889eacef17,6b2dc32e-4414-48e0-9472-ebb49a766c28
3,R38W5,approved,990421d0-2287-4a8c-824a-cbf732199c33,"{'type': 'Polygon', 'coordinates': [[[32.83320...",2024-04-01,2024-07-31,tree-planting,agroforest,full,NaN,0.184905,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2023-01-31T00:00:00.000Z', 'submit...",e489aba2-b89a-455e-9436-05275bbff778,6b2dc32e-4414-48e0-9472-ebb49a766c28
4,R33W12,approved,1373d39d-7cca-4382-8518-b57a06df9a02,"{'type': 'Polygon', 'coordinates': [[[32.58095...",2024-06-01,2024-11-30,tree-planting,agroforest,full,240.0,0.264170,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2022-09-30T00:00:00.000Z', 'submit...",112e1fd2-60bd-4384-a3f7-67ae9cbcbbcd,6b2dc32e-4414-48e0-9472-ebb49a766c28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15698,Grace Mensah,approved,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.24163...",2024-07-19,2024-07-19,tree-planting,agroforest,single-line,15.0,1.424275,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",2bbc9aae-5b21-470b-bd35-9edf5efcd7e6,24e290f1-5b13-4ca8-acce-43dcc8e5bb43
15699,Kwame Amponsem,approved,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.25119...",2024-08-08,2024-08-08,tree-planting,agroforest,single-line,40.0,0.981553,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",ab8af29f-9ac8-4624-9902-580c51c908ff,24e290f1-5b13-4ca8-acce-43dcc8e5bb43
15700,Dora Agyiri,approved,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.25262...",2024-07-24,2024-07-24,tree-planting,agroforest,single-line,20.0,1.203820,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",fcb25d3f-990a-4a08-922a-08ee17d35c22,24e290f1-5b13-4ca8-acce-43dcc8e5bb43
15701,Justice Amoah,approved,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.25093...",2024-07-11,2024-07-11,tree-planting,agroforest,single-line,40.0,0.981553,"[{'indicatorSlug': 'restorationByStrategy', 'y...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",02713edf-8615-491f-b8fe-20dfd13d9ddd,24e290f1-5b13-4ca8-acce-43dcc8e5bb43


In [32]:
project_df = pd.DataFrame(project_results)
project_df.columns = project_df.columns.str.lower()

project_df.to_csv('../data/tm_api_021025.csv', index=False)
project_df.to_csv('/Users/jessica.ertel/github/maxar-tools/data/tm_api_021025.csv', index=False)